In [27]:
! pip install pytorch-lightning --quiet
from google.colab import drive
drive.mount('/content/drive/')

%cd ./drive/MyDrive/Colab\ Notebooks/nn_assign4_lstm

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
[Errno 2] No such file or directory: './drive/MyDrive/Colab Notebooks/nn_assign4_lstm'
/content/drive/MyDrive/Colab Notebooks/nn_assign4_lstm


In [28]:
from util import *
from torch.utils.data import random_split, DataLoader
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,RobustScaler
log_dir = "lightning_logs"
seed_everything(777)

Global seed set to 777


777

In [29]:
def load_data(test=False, seed = 12345):
    train_x = np.load('./Data/train_data.npy') # 11 ppl
    train_y = np.load('./Data/train_label.npy')
    test_x = np.load('./Data/test_data.npy') # 4 ppl
    test_y = np.load('./Data/test_label.npy')
    scaler = MinMaxScaler()
    train_x = scaler.fit_transform(train_x)
    test_x = scaler.transform(test_x)
    if(test):
        return test_x,test_y
    else:
       return train_x,train_y
    
x,y = load_data()
print(x.shape)
print(y.shape)
# x

(37367, 310)
(37367,)


In [30]:
# show data
class SeedDataset(Dataset):
    def __init__(self,window,test=False,):
        self.data,self.target = load_data(test)
        self.window = window
        self.target = np.array(self.target,int)
        index = np.where(self.target==-1)
        self.target[index]=2 
    def __len__(self):
        return len(self.target) // self.window

    def __getitem__(self,idx):
        idx = idx * self.window
        data = torch.tensor(self.data[idx:idx+self.window], dtype=torch.float)
        label_ls = np.bincount(self.target[idx:idx+self.window]).argmax()
        
        label = torch.tensor(label_ls,dtype=torch.long)
        return data, label
    


In [31]:
sdm = SeedDataset(window=10)
print(f'training data {len(sdm)}')
idx= 1000
# np.unique(sdm.target)
print(f'x shape:{sdm[idx][0].shape}')
# print(f'y value:{sdm[idx][1]}')
# np.unique([sdm[idx][1],sdm[idx+1][1],sdm[idx+20][1],sdm[idx+30][1],sdm[idx+1000][1],sdm[idx+5][1]])

training data 3736
x shape:torch.Size([10, 310])


In [32]:

class SeedDataModule(pl.LightningDataModule):

    def __init__(self,window=10,batch_size=32,shuffle=False):
        super().__init__()
        self.batch_size=batch_size
        self.window = window
        self.shuffle = shuffle
    def setup(self,stage=None):
        if stage == 'fit' or stage is None:
            train_full = SeedDataset(self.window)
            train, val = train_test_split(train_full,shuffle=self.shuffle)
            self.train_set = train_full
            # self.val_set = val
        
        if stage == 'test' or stage is None:
            self.test_set = SeedDataset(self.window,test=True)
        

    def train_dataloader(self):
        return DataLoader(self.train_set,batch_size=self.batch_size,shuffle=self.shuffle)

    # def val_dataloader(self):
    #     return DataLoader(self.val_set,batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set,batch_size=1)

In [33]:
class LSTM(LightningModule):
    def __init__(self,hidden_size = 100,window_size=10, lr=1e-3,batch_size=16):
        super().__init__()
        self.save_hyperparameters()
        self.lstm = nn.LSTM(input_size= 310,
                    hidden_size = self.hparams.hidden_size,
                    num_layers = 1,
                    batch_first=True,
                    bidirectional=False)
        self.criterion = nn.CrossEntropyLoss()
        self.fc = nn.Linear(in_features = self.hparams.hidden_size, out_features=3)
        self.target = None
        self.pred = None

    def forward(self,x):
        # x = 32,10,310 -> batch_size, seq_length, input_size
        h0 = torch.zeros(1,x.size(0),self.hparams.hidden_size, device=self.device)
        c0 = torch.zeros(1,x.size(0),self.hparams.hidden_size, device=self.device)
       
        # print(x.shape)
        out, _ = self.lstm(x,(h0,c0)) # out: batch_size, seq_length, hidden_size
        
        out = out[:,-1,:]

        return self.fc(out) 

    def training_step(self,batch,batch_idx):
        loss = 0
        x, label = batch
        # y = torch.squeeze(label,0)
   
        # print(label.shape)
        logits = self.forward(x)
        loss += self.criterion(logits, label)
        _, predicted = torch.max(logits, 1)
        # print(predicted)
        acc = accuracy(predicted,label)

        self.log('train/loss', loss,on_step=False, on_epoch=True)
        self.log('train/acc',acc, on_step=False, on_epoch=True)
        return loss



    def test_step(self, batch, batch_idx):
        # validation_step defined the train loop.
        # It is independent of forward

        x, class_label = batch
        # print(x)
        # classfication loss
        logits = self.forward(x)
        score, predicted = torch.max(logits, 1)
        # print(f'predicted:{predicted.item()}, should be {class_label.item()}')
        acc = accuracy(predicted,class_label)
        
        return {"acc":acc,
                "pred":predicted,
                "label":class_label}

    def test_epoch_end(self,outputs):
        test_acc = torch.mean(
            torch.stack([x["acc"] for x in outputs]))
        self.test_acc=test_acc
        self.log("hp_metric",test_acc)
        preds = torch.stack([x["pred"] for x in outputs])
        targets = torch.stack([x["label"] for x in outputs])
        self.target =targets
        self.pred = preds
        # self.learned_weights = list(self.parameters())
        # print(classification_report(y_test, res, target_names=target_names))
        # confusion_matrix = pl.metrics.functional.confusion_matrix(preds, targets, num_classes=10)
        # df_cm = pd.DataFrame(confusion_matrix.to("cpu").numpy(), index = range(10), columns=range(10))
        # plt.figure(figsize = (10,7))
        # fig_ = sns.heatmap(df_cm, annot=True, cmap='Spectral').get_figure()
        # plt.close(fig_)
        
        # self.logger.experiment.add_figure("Confusion matrix", fig_, self.current_epoch)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr,weight_decay=2e-5)
                                    
        return optimizer

In [34]:
# model_accuracy_lstm = list()
def train_model(lr=1e-4,window = 10,batch_size=16,hidden_size=100):
    logger = TensorBoardLogger(log_dir, name="lstm")
    trainer = Trainer(max_epochs=12, progress_bar_refresh_rate=0,
                      fast_dev_run=False,
                        logger= logger,
                    # gpus=-1,
                    weights_summary=None
                    )
    m = LSTM(lr=lr,hidden_size=hidden_size,window_size = window,batch_size=batch_size)
    sdm = SeedDataModule(window=window,batch_size=batch_size,shuffle=False)
    trainer.fit(m,sdm)
    trainer.test(m,datamodule=sdm)
    return m



# train_model(window = 10,lr=1e-2,batch_size=16,hidden_size=100)
# train_model(window = 10,lr=1e-3,batch_size=16,hidden_size=100)
# train_model(window = 10,lr=1e-2,batch_size=16,hidden_size=100)
# train_model(window = 10,lr=1e-2,batch_size=16,hidden_size=100)
# train_model(window = 10,lr=1e-3,batch_size=32,hidden_size=100)
# train_model(window = 10,lr=1e-4,batch_size=64,hidden_size=100)
# train_model(window = 10,lr=1e-5,batch_size=10,hidden_size=128) #45%
# train_model(window = 8,lr=1e-3,batch_size=10,hidden_size=128) #53%
m = train_model(window = 50,lr=1e-3,batch_size=10,hidden_size=128) #54%
# train_model(window = 40,lr=1e-3,batch_size=10,hidden_size=128) #42%
# train_model(window = 30,lr=1e-3,batch_size=10,hidden_size=128) #51%
# train_model(window = 20,lr=1e-3,batch_size=10,hidden_size=128) #51%
# train_model(window = 70,lr=1e-3,batch_size=10,hidden_size=128) #44.8%
# train_model(window = 80,lr=1e-3,batch_size=10,hidden_size=128) #42%

window_ = [10,1]
lr_ = [1e-3]
batch_ = [1]
# for window in window_:
#     for lr in lr_:
#         for batch in batch_:
#             train_model(window = window,lr=lr,batch_size=batch,hidden_size=128)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'hp_metric': 0.5904058814048767}
--------------------------------------------------------------------------------


In [35]:
target_names = ["0","1","-1"]
from sklearn.metrics import classification_report
print(classification_report(m.pred, m.target, target_names=target_names))
# confusion_matrix = pl.metrics.functional.confusion_matrix(preds, targets, num_classes=10)

              precision    recall  f1-score   support

           0       0.20      0.67      0.31        27
           1       0.94      0.56      0.70       160
          -1       0.60      0.62      0.61        84

    accuracy                           0.59       271
   macro avg       0.58      0.62      0.54       271
weighted avg       0.76      0.59      0.64       271



In [36]:
# # !kill 287
# %load_ext tensorboard
# %tensorboard --logdir lightning_logs/